# Proceso de ETL del DataSet steam_games

In [105]:
#Importo las librerias que voy a usar
import pandas as pd
import gzip
import ast

In [106]:
# Tengo que cargar el dataset que voy a utilizar
games = pd.read_json("../Dataset/steam_games.json.gz", compression="gzip", lines=True)
games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se debera detectar la cantidad de valores nulos y se debera proceder a eliminarlos

In [107]:
# Con la siguiente funcion elimino las filas que solo tienen valores nulos
games = games.dropna(how='all')
# Y comprobamos si funciona
games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com


In [108]:
# Ahora identifico la cantidad de valores nulos que qudaron en el dataset
for column in games.columns:
    print(f'Hay {games[column].isna().sum()} valores nulos en {column}')

Hay 8052 valores nulos en publisher
Hay 3283 valores nulos en genres
Hay 2 valores nulos en app_name
Hay 2050 valores nulos en title
Hay 0 valores nulos en url
Hay 2067 valores nulos en release_date
Hay 163 valores nulos en tags
Hay 2 valores nulos en reviews_url
Hay 670 valores nulos en specs
Hay 1377 valores nulos en price
Hay 0 valores nulos en early_access
Hay 2 valores nulos en id
Hay 3299 valores nulos en developer


In [109]:
# Sabiendo la cantidad de nulos que quedaron, ahora procedo a eliminarlos 
games = games.dropna() 
#Ahora hago el checkeo 
for column in games.columns:
    print(f'Hay {games[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en publisher
Hay 0 valores nulos en genres
Hay 0 valores nulos en app_name
Hay 0 valores nulos en title
Hay 0 valores nulos en url
Hay 0 valores nulos en release_date
Hay 0 valores nulos en tags
Hay 0 valores nulos en reviews_url
Hay 0 valores nulos en specs
Hay 0 valores nulos en price
Hay 0 valores nulos en early_access
Hay 0 valores nulos en id
Hay 0 valores nulos en developer


Borro las columnas que no voy a utilizar

In [110]:
games.drop(columns= ['app_name', 'publisher', 'url', 'reviews_url', 'early_access', 'specs'], inplace=True)

In [111]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        22530 non-null  object 
 1   title         22530 non-null  object 
 2   release_date  22530 non-null  object 
 3   tags          22530 non-null  object 
 4   price         22530 non-null  object 
 5   id            22530 non-null  float64
 6   developer     22530 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.4+ MB


In [112]:
games.head(2)

,genres,title,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL


Modifico la columna price para convertir los precios en float

In [113]:
# Identifico que datos hay dentro de la columna
games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 18.99, 29.99,
       10.99, 2.99, 1.5899999999999999, 14.99, 1.99, 59.99, 9.99, 6.99,
       7.99, 39.99, 'Free', 19.99, 7.49, 8.99, 12.99, 5.99, 2.49, 15.99,
       1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 13.98, 149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 'Free Mod',
       54.99, 64.99, 74.99, 0.89, 0.5, 'Play Now', 299.99, 1.29, 119.99,
       44.99, 3.0, 15.0, 1.3900000000000001, 'Free HITMAN™ Holiday Pack',
       2.0, 4.0, 1.9500000000000002, 1.5, 6.66, 26.99, 399.99, 31.99,
       20.0, 40.0, 5.0, 3.33, 38.85, 71.7, 995.0, 5.49, 27.49, 3.39, 6.0,
       19.95, 20.99, 499.99, 27.99, 199.99, 4.68, 131.4, 44.98, 202.76,
       2.3, 0.9500000000000001, 36.99, 172.24, 249.99, 2.9699999999999998,
       10.96, 2.66, 6.48, 10.0, 1.0, 11.15, 'Play the Demo', 

In [114]:
def price_to_float(price):
    # Si el precio es un string, su valor será 0.
    if type(price) == str:
        return float(0)
    # Si el precio no es un string, lo convertimos a float.
    else:
        return float(price)


# Convertir los precios de los juegos a formato float usando la función 'price_to_float'
games['price'] = games['price'].apply(price_to_float)

In [115]:
#pd.set_option('display.float_format', '{:.2f}'.format)
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        22530 non-null  object 
 1   title         22530 non-null  object 
 2   release_date  22530 non-null  object 
 3   tags          22530 non-null  object 
 4   price         22530 non-null  float64
 5   id            22530 non-null  float64
 6   developer     22530 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.4+ MB


In [116]:
#Convierto todas las columnas de tipo 'object' a 'string' 
for column in games.select_dtypes(include=['object']).columns:
    games[column] = games[column].astype(str)


#Este codigo no fue utilizado a la hora de hacer el eda o el modelo de recomendacion

# Revisamos los cambios 
games.head(5)

,genres,title,release_date,tags,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,643980.0,Secret Level SRL
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",0.00,670290.0,Poolians.com
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",3.99,772540.0,Trickjump Games Ltd


In [117]:
#Convierto la columna release_date a datetime
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')

In [118]:
#Convierto id en un string para utilizarlo para unir las tablas
games['id'] = games['id'].astype(str).str.split('.').str[0]

In [119]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        22530 non-null  object        
 1   title         22530 non-null  object        
 2   release_date  22496 non-null  datetime64[ns]
 3   tags          22530 non-null  object        
 4   price         22530 non-null  float64       
 5   id            22530 non-null  object        
 6   developer     22530 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.4+ MB


Reducir el archivo para que la carga del render no se me complique 

In [120]:
games.shape

(22530, 7)

In [121]:
# Genero un nuevo oreden para las columnas y luego procedo a convertir las columnas de generos y tags a strings
new_order = ['id', 'title', 'developer', 'release_date', 'genres', 'tags', 'price']

games = games[new_order]
games.head(5)

,id,title,developer,release_date,genres,tags,price
88310,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99
88311,643980,Ironbound,Secret Level SRL,2018-01-04,"['Free to Play', 'Indie', 'RPG', 'Strategy']","['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00
88312,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Simulation', 'Sports', 'Casu...",0.00
88313,767400,弹炸人2222,彼岸领域,2017-12-07,"['Action', 'Adventure', 'Casual']","['Action', 'Adventure', 'Casual']",0.99
88315,772540,Battle Royale Trainer,Trickjump Games Ltd,2018-01-04,"['Action', 'Adventure', 'Simulation']","['Action', 'Adventure', 'Simulation', 'FPS', '...",3.99


In [274]:
# Exportamos el dataset con un nuevo formato de archivo, que en este caso vendria a ser parquet con una compresion de tipo snappy
games.to_parquet('../Dataset/games_clean.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

# ETL para el sistema de recomendacion de juegos

### Para el Sistema de Recoendacion de juegos
Debere trabajar con algunas columnas del DataSet como:
* id
* title
* release_date
* genres
## Entonces las columnas que no usare las eliminare

In [64]:
games.columns

Index(['id', 'title', 'developer', 'release_date', 'genres', 'tags', 'price'], dtype='object')

Vamos a trabajar con la columna "genres"

In [65]:
# Observamos la composicion y vemos que es una lista
games['genres']

88310         [Action, Casual, Indie, Simulation, Strategy]
88311                  [Free to Play, Indie, RPG, Strategy]
88312     [Casual, Free to Play, Indie, Simulation, Sports]
88313                           [Action, Adventure, Casual]
88315                       [Action, Adventure, Simulation]
                                ...                        
120439                   [Action, Adventure, Casual, Indie]
120440                [Casual, Indie, Simulation, Strategy]
120441                            [Casual, Indie, Strategy]
120442                          [Indie, Racing, Simulation]
120443                                      [Casual, Indie]
Name: genres, Length: 22530, dtype: object

In [104]:
games.to_parquet('../Dataset/games_sistem.parquet', compression='snappy', index=False)